In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, filters
import aicsimageio
from aicsimageio import AICSImage
import seaborn as sns
import pandas as pd
from sklearn.mixture import GaussianMixture
from matplotlib.patches import PathPatch
from matplotlib.path import Path
import matplotlib.patches as patches

In [ ]:
#Function to plot protein expression
#File is the AICS file containing the data
#sig1 is the sigma value for the Gaussian filter for dapi
#sig2 is the sigma value for the Gaussian filter for TdTomato
#cut1 is the expression cutoff for dapi
#cut2 is the expression cutoff for TdTomato
#r1 and r2 are 2 element lists that define the rectangular ROI
#One or both can also be zero if you do not want to define boundaries for that part
#back1 and back2 are used to define a suitable background region containing no dapi signal

def plot(file, sig1, sig2, cut1, cut2, r1, r2, back1, back2, plot = True):
    #Read in the image and extract the channels
    img = AICSImage(file)
    goi = img.get_image_data("ZYX", C=0, S=0, T=0)[0]
    tdt = img.get_image_data("ZYX", C=1, S=0, T=0)[0]
    nes = img.get_image_data("ZYX", C=2, S=0, T=0)[0]
    dapi = img.get_image_data("ZYX", C=3, S=0, T=0)[0]
    
    #If we want to plot, make a plot of the image including the ROI
    if plot:
        fig, ax = plt.subplots()
        ax.imshow(tdt)
        rect = patches.Rectangle((r2[0], r1[0]), r2[1] - r2[0], r1[1] - r1[0], linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        plt.show()
    
    #plt.imshow(tdt)
    #plt.show()
    #plt.imshow(goi)
    #plt.show()
    #plt.imshow(dapi)
    #plt.show()
    
    #Apply Gaussian filter
    dapiFilt = filters.gaussian(dapi, sigma=sig1)
    tdtFilt = filters.gaussian(tdt, sigma=sig2)
    #plt.imshow(tdtFilt)
    #plt.show()
    #plt.imshow(dapiFilt)
    #plt.show()
    
    #Apply mask
    dapiMask = dapiFilt > cut1
    tdtMask = tdtFilt > cut2
    
    #Plot if desired
    if plot:
        plt.imshow(tdtMask)
        plt.show()
        plt.imshow(dapiMask)
        plt.show()
    if plot:
        flat_tdt = tdtFilt.flatten()
        dist = plt.hist(flat_tdt[np.where(flat_tdt > 0)], density = True, color = "magenta", alpha = 0.5, bins = 1000, cumulative = False)
        plt.show()
    
    #Compute the nestin profile
    #Tdt+ regions are donor, Tdt- are host
    nes_donor = (nes+1)*dapiMask*tdtMask
    nes_host = (nes+1)*dapiMask*~tdtMask
    
    #Compute profile for gene of interest
    goi_donor = (goi+1)*dapiMask*tdtMask
    goi_host = (goi+1)*dapiMask*~tdtMask
    
    #Plot in the specified region
    #If both are zero, then want for whole image
    if r1 == 0 and r2 == 0:
        nes_host_flat = nes_host.flatten()
        nes_donor_flat = nes_donor.flatten()
        goi_host_flat = goi_host.flatten()
        goi_donor_flat = goi_donor.flatten()
    
    #If just r1 is zero, then it is just a stripe
    elif r1 == 0:
        nes_host_flat = nes_host[:, r2[0]:r2[1]].flatten()
        nes_donor_flat = nes_donor[:, r2[0]:r2[1]].flatten()
        goi_host_flat = goi_host[:, r2[0]:r2[1]].flatten()
        goi_donor_flat = goi_donor[:, r2[0]:r2[1]].flatten()
    
    #Also stripe for r2
    elif r2 == 0:
        nes_host_flat = nes_host[r1[0]:r1[1], :].flatten()
        nes_donor_flat = nes_donor[r1[0]:r1[1], :].flatten()
        goi_host_flat = goi_host[r1[0]:r1[1], :].flatten()
        goi_donor_flat = goi_donor[r1[0]:r1[1], :].flatten()
        
    #Otherwise, we want everything in the well-defined rectangle
    else:
        nes_host_flat = nes_host[r1[0]:r1[1], r2[0]:r2[1]].flatten()
        nes_donor_flat = nes_donor[r1[0]:r1[1], r2[0]:r2[1]].flatten()
        goi_host_flat = goi_host[r1[0]:r1[1], r2[0]:r2[1]].flatten()
        goi_donor_flat = goi_donor[r1[0]:r1[1], r2[0]:r2[1]].flatten()
    
    #Get only non-zero pixels
    nes_host_flat = nes_host_flat[np.where(nes_host_flat > 0)]
    nes_donor_flat = nes_donor_flat[np.where(nes_donor_flat > 0)]
    
    goi_host_flat = goi_host_flat[np.where(goi_host_flat > 0)]
    goi_donor_flat = goi_donor_flat[np.where(goi_donor_flat > 0)]
    
    #Flip the host and donor colors to get them to match the right species
    if "M-R" in file:
        host_color = "green"
        donor_color = "magenta"
    elif "R-M" in file:
        host_color = "magenta"
        donor_color = "green"
    if plot:
        #Plot histogram of expression for Nestin
        plt.hist(np.log10(nes_donor_flat), density = True, color = donor_color, alpha = 0.5, bins = 100)
        plt.hist(np.log10(nes_host_flat), density = True, color = host_color, alpha = 0.5, bins = 100)
        plt.title("Nestin expression")
        plt.show()
        
        #Do the same for the gene of interest
        plt.hist(np.log10(goi_donor_flat), density = True, color = donor_color, alpha = 0.5, bins = 100)
        plt.hist(np.log10(goi_host_flat), density = True, color = host_color, alpha = 0.5, bins = 100)
        plt.title(file.split("/")[2].replace(".czi", "") + " expression")
        plt.show()
        
        #Print the median value
        print(np.median(goi_host_flat))
    
    #Compute background subtraction
    goi_back = np.mean(goi[back1:, 0:back2].flatten() + 1)
    
    #Return background subtracted arrays for the gene of interest
    return goi_host_flat-goi_back, goi_donor_flat-goi_back

#Plot across all conditions
#Takes as input the arrays returned by plot defined above
def plot_across(mr_host, mr_donor, rm_host, rm_donor, gene):
    
    #Get only non-zero ones and log transform
    #Then plot histogram of pixel values
    plt.hist(np.log10(mr_host[np.where(mr_host > 1)]), density = True, color = "green", alpha = 0.5, bins = 100)
    plt.hist(np.log10(rm_host[np.where(rm_host > 1)]), density = True, color = "magenta", alpha = 0.5, bins = 100)
    plt.title(gene + " expression hosts")
    plt.show()
    plt.hist(np.log10(mr_donor[np.where(mr_donor > 1)]), density = True, color = "magenta", alpha = 0.5, bins = 100)
    plt.hist(np.log10(rm_donor[np.where(rm_donor > 1)]), density = True, color = "green", alpha = 0.5, bins = 100)
    plt.title(gene + " expression donors")
    plt.show()

In [ ]:
#Iterate through computing based on different masks
#Designed to make sure that the estimates are not sensitive to the parameters chosen

#Define masks for RM and MR images
rm_dapi_masks = [0.08, 0.1, 0.12]
rm_tdt_masks = [0.05, 0.06, 0.07]
mr_dapi_masks = [0.1, 0.12, 0.14]
mr_tdt_masks = [0.1, 0.11, 0.12]
out = []

#Iterate through all combinations
for rm_dapi_mask in rm_dapi_masks:
    for rm_tdt_mask in rm_tdt_masks:
        for mr_dapi_mask in mr_dapi_masks:
            for mr_tdt_mask in mr_tdt_masks:
                
                #Can change file name to the correct file
                rm_host, rm_donor = plot("Batch3/R-M Ch8/c-Jun.czi", 10, 5, rm_dapi_mask, rm_tdt_mask, [800, 1600], [1400, 3300], 2750, 1000, plot = False)
                mr_host, mr_donor = plot("Batch3/M-R Ch1/c-Jun.czi", 10, 5, mr_dapi_mask, mr_tdt_mask, [1000, 3000], [2750, 5000], 4000, 1000, plot = False)
                
                #Compute mean non-zero log2 transformed expression
                mr_host_mean = np.mean(np.log2(mr_host[np.where(mr_host > 1)]))
                mr_donor_mean = np.mean(np.log2(mr_donor[np.where(mr_donor > 1)]))
                rm_host_mean = np.mean(np.log2(rm_host[np.where(rm_host > 1)]))
                rm_donor_mean = np.mean(np.log2(rm_donor[np.where(rm_donor > 1)]))
                
                #Append
                out.append([rm_dapi_mask, rm_tdt_mask, mr_dapi_mask, mr_tdt_mask, mr_donor_mean - mr_host_mean, rm_host_mean - rm_donor_mean, rm_host_mean - mr_host_mean, mr_donor_mean - rm_donor_mean])
df = pd.DataFrame(out)
#Make column names
df.columns = ["RM_DAPI_mask", "RM_TdT_mask", "MR_DAPI_mask", "MR_TdT_mask", "DM/HR", "HM/DR", "HM/HR", "DM/DR"]

#Write out, can change file name
df.to_csv("Batch3_RM-Ch8_MR-Ch1_c-Jun.csv", sep = ",", index = False)

In [ ]:
#Print the standard errors
#Low standard error suggests that it is not sensitive to our choice of parameters
v = pd.read_csv("Batch3_RM-Ch8_MR-Ch1_c-Jun.csv", sep = ",")
print(np.sqrt(np.var(v["HM/HR"]))/9)
print(np.sqrt(np.var(v["DM/HR"]))/9)
print(np.sqrt(np.var(v["DM/DR"]))/9)

v

In [ ]:
#Actual parameters used for Jun RM
rm_host, rm_donor = plot("Batch3/R-M Ch8/c-Jun.czi", 10, 5, 0.1, 0.07, [800, 1600], [1400, 3300], 2750, 1000, plot = True)

In [ ]:
#Actual parameters used for Jun MR
mr_host, mr_donor = plot("Batch3/M-R Ch1/c-Jun.czi", 10, 5, 0.14, 0.12, [1000, 3000], [2750, 5000], 4000, 1000)


In [ ]:
#Plot everything to check it is okay
plot_across(mr_host, mr_donor, rm_host, rm_donor, "c-Jun")

#Compute mean of log2 transformed non-zero data
mr_host_mean = np.mean(np.log2(mr_host[np.where(mr_host > 1)]))
mr_donor_mean = np.mean(np.log2(mr_donor[np.where(mr_donor > 1)]))
rm_host_mean = np.mean(np.log2(rm_host[np.where(rm_host > 1)]))
rm_donor_mean = np.mean(np.log2(rm_donor[np.where(rm_donor > 1)]))

#Print the log2 fold-changes needed to compute what we want
print("DM/HR", mr_donor_mean - mr_host_mean)
print("HM/DR", rm_host_mean - rm_donor_mean)
print("HM/HR", rm_host_mean - mr_host_mean)
print("DM/DR", mr_donor_mean - rm_donor_mean)

In [ ]:
#Iterate through computing based on different masks,this time for batch1 of Jun measurements
rm_dapi_masks = [0.09, 0.1, 0.11]
rm_tdt_masks = [0.035, 0.04, 0.045]
mr_dapi_masks = [0.16, 0.18, 0.20]
mr_tdt_masks = [0.11, 0.13, 0.15]
out = []
for rm_dapi_mask in rm_dapi_masks:
    for rm_tdt_mask in rm_tdt_masks:
        for mr_dapi_mask in mr_dapi_masks:
            for mr_tdt_mask in mr_tdt_masks:
                rm_host, rm_donor = plot("Batch1/R-M Ch2/c-Jun.czi", 10, 5, rm_dapi_mask, rm_tdt_mask, [800, 1600], [1400, 3300], 2750, 1000, plot = False)
                mr_host, mr_donor = plot("Batch1/M-R Ch5/c-Jun.czi", 10, 5, mr_dapi_mask, mr_tdt_mask, [1000, 3000], [2750, 5000], 4000, 1000, plot = False)
                
                mr_host_mean = np.mean(np.log2(mr_host[np.where(mr_host > 1)]))
                mr_donor_mean = np.mean(np.log2(mr_donor[np.where(mr_donor > 1)]))
                rm_host_mean = np.mean(np.log2(rm_host[np.where(rm_host > 1)]))
                rm_donor_mean = np.mean(np.log2(rm_donor[np.where(rm_donor > 1)]))
                
                out.append([rm_dapi_mask, rm_tdt_mask, mr_dapi_mask, mr_tdt_mask, mr_donor_mean - mr_host_mean, rm_host_mean - rm_donor_mean, rm_host_mean - mr_host_mean, mr_donor_mean - rm_donor_mean])
df = pd.DataFrame(out)
df.columns = ["RM_DAPI_mask", "RM_TdT_mask", "MR_DAPI_mask", "MR_TdT_mask", "DM/HR", "HM/DR", "HM/HR", "DM/DR"]
df.to_csv("Batch1_RM-Ch2_MR-Ch5_c-Jun.csv", sep = ",", index = False)

In [ ]:
#Print Standard deviations again
v = pd.read_csv("Batch1_RM-Ch2_MR-Ch5_c-Jun.csv", sep = ",")
print(np.sqrt(np.var(v["HM/HR"]))/9)
print(np.sqrt(np.var(v["DM/HR"]))/9)
print(np.sqrt(np.var(v["DM/DR"]))/9)

In [ ]:
#Repeat above for batch1 values
mr_host, mr_donor = plot("Batch1/M-R Ch5/c-Jun.czi", 10, 5, 0.16, 0.11, [100, 1000], [4300, 5800], 3000, 1000)

rm_host, rm_donor = plot("Batch1/R-M Ch2/c-Jun.czi", 10, 5, 0.11, 0.04, [1300, 2250], [2500, 3500], 2000, 1000)

mr_host_mean = np.mean(np.log2(mr_host[np.where(mr_host > 1)]))
mr_donor_mean = np.mean(np.log2(mr_donor[np.where(mr_donor > 1)]))
rm_host_mean = np.mean(np.log2(rm_host[np.where(rm_host > 1)]))
rm_donor_mean = np.mean(np.log2(rm_donor[np.where(rm_donor > 1)]))

print(mr_donor_mean - mr_host_mean)
print(rm_host_mean - rm_donor_mean)
print(rm_host_mean - mr_host_mean)
print(mr_donor_mean - rm_donor_mean)

In [ ]:
#Now do computations for Hspa5 as done for Jun
mr_host, mr_donor = plot("Batch1/M-R Ch5/Hspa5.czi", 10, 5, 0.2, 0.15, [150, 1100], [4400, 5700], 3000, 1000)
rm_host, rm_donor = plot("Batch1/R-M Ch2/Hspa5.czi", 10, 5, 0.15, 0.032, [1300, 2250], [1250, 3500], 2000, 1000)

plot_across(mr_host, mr_donor, rm_host, rm_donor, "Hspa5")
mr_host_mean = np.mean(np.log2(mr_host[np.where(mr_host > 1)]))
mr_donor_mean = np.mean(np.log2(mr_donor[np.where(mr_donor > 1)]))
rm_host_mean = np.mean(np.log2(rm_host[np.where(rm_host > 1)]))
rm_donor_mean = np.mean(np.log2(rm_donor[np.where(rm_donor > 1)]))

print("DM/HR", mr_donor_mean - mr_host_mean)
print("HM/DR", rm_host_mean - rm_donor_mean)
print("HM/HR", rm_host_mean - mr_host_mean)
print("DM/DR", mr_donor_mean - rm_donor_mean)

In [ ]:
#Can also similarly compute for Hspa5 Batch 3
mr_host, mr_donor = plot("Batch3/M-R Ch1/Hspa5.czi", 10, 5, 0.2, 0.2, [1000, 3000], [2750, 5000], 4000, 1000)
rm_host, rm_donor = plot("Batch3/R-M Ch8/Hspa5.czi", 10, 5, 0.1, 0.045, [750, 1500], [1250, 3750], 2750, 1000)
plot_across(mr_host, mr_donor, rm_host, rm_donor, "Hspa5")
mr_host_mean = np.mean(np.log2(mr_host[np.where(mr_host > 1)]))
mr_donor_mean = np.mean(np.log2(mr_donor[np.where(mr_donor > 1)]))
rm_host_mean = np.mean(np.log2(rm_host[np.where(rm_host > 1)]))
rm_donor_mean = np.mean(np.log2(rm_donor[np.where(rm_donor > 1)]))

print("DM/HR", mr_donor_mean - mr_host_mean)
print("HM/DR", rm_host_mean - rm_donor_mean)
print("HM/HR", rm_host_mean - mr_host_mean)
print("DM/DR", mr_donor_mean - rm_donor_mean)